In [7]:

from time import time

import numpy as np
import pandas as pd
import sklearn.metrics as metrics
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection

from sklearn.cluster import KMeans

In [8]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_)))
#              metrics.silhouette_score(data, estimator.labels_,
#                                       metric='euclidean',
#                                       sample_size=sample_size)))

In [10]:
trainSet = pd.read_csv('dota2Train.csv')
testSet = pd.read_csv('dota2Test.csv')

x = trainSet.drop('-1', 1)
y = trainSet['-1']
testX = testSet.drop('-1', 1)
testY = testSet['-1']

np.random.seed(42)

n_samples, n_features = x.shape
n_classes = len(np.unique(y))
labels = y
sample_size = n_samples * 0.16
print n_samples
print n_features
print n_classes

92649
116
2


In [12]:
print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_classes, n_samples, n_features))
print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')
bench_k_means(KMeans(init='k-means++', n_clusters=n_classes, n_init=10, max_iter = 10000),
              name="k-means++", data=x)

bench_k_means(KMeans(init='random', n_clusters=n_classes, n_init=10, max_iter = 10000),
              name="random", data=x)
pca = PCA(n_components=n_classes).fit(x)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_classes, n_init=1, max_iter = 10000),
              name="PCA-based",
              data=x)
ica = FastICA(n_components=n_classes).fit(x)
bench_k_means(KMeans(init=ica.components_, n_clusters=n_classes, n_init=1, max_iter = 10000),
              name="ICA-based",
              data=x)
rp = GaussianRandomProjection(n_components=n_classes).fit(x)
bench_k_means(KMeans(init=rp.components_, n_clusters=n_classes, n_init=1, max_iter = 10000),
              name="RP-based",
              data=x)
print(82 * '_')

n_digits: 2, 	 n_samples 92649, 	 n_features 116
__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	2.36s	21770909	0.000	0.000	0.000	-0.000	-0.000
random   	2.81s	21770909	0.000	0.000	0.000	-0.000	-0.000
PCA-based	0.52s	21770909	0.000	0.000	0.000	-0.000	-0.000
ICA-based	0.48s	21770909	0.000	0.000	0.000	-0.000	-0.000
RP-based 	0.44s	21770909	0.000	0.000	0.000	-0.000	-0.000
__________________________________________________________________________________
